### Exercises Week 3 

In [1]:
# %% imports
# libraries
import torch
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
import os
import numpy as np

# local imports
import Fast_MRI_dataloader
# import autoencoder_template
from matplotlib import pyplot as plt
from Fast_MRI_dataloader import create_dataloaders

# set-up folders
data_loc = os.getcwd()
print(data_loc)
data_loc = data_loc + '\Fast_MRI_Knee'
batch_size = 16
train_loader, test_loader = create_dataloaders(data_loc, batch_size)

C:\Users\20167271\Desktop\ML for signal processing\5LSL0-GIT\Assignment 4


In [2]:
def acc_img(X):
    '''
    Given the partial k_space image X, calculate the accelerated measurement image:

    Arguments:
    ----------------
    X: torch.tensor()

    returns:
    --------------
    Y: torch.tensor()
    '''
    Y = torch.fft.ifft2(torch.fft.ifftshift(X, dim=(-2,-1)))
    return Y

In [3]:
def K_space(gt):
    ''' Return the K-space of an input image:
        Arguments:
        ---------------------
        X: torch.Tensor (default: -)

        Returns:
        ---------------------------
        KS: torch.Tensor
    '''
    # KS = torch.log(torch.abs(torch.fft.fftshift(torch.fft.fft2(gt)))) 
    KS = torch.fft.fftshift(torch.fft.fft2(gt))
    return KS

impulse=torch.zeros((320,320))
impulse[160:162,160:162] = 1
A = torch.abs(K_space(impulse))

In [ ]:
plt.imshow(torch.abs(A).numpy(), cmap='gray')

## Exercise 5 - ConvNet
As a starting point for a deep learning solution, design an end-to-end convolutional neural
network. As input it should take an initial reconstruction from a partial k-space measurement
and output a guess for the final reconstruction. The net is not (yet) allowed to make
use of knowledge about the sampling mask.

(a) [1pt] What loss function would be most suitable to train this network, and what
underlying assumption have you made by choosing this loss function?

KL divergence is a metric to compare the distribution of pixel values between the input and output images. The MSE loss can be suitable as well but we already know that there is loss of spatial information given the mask and we do not want to penalize shifts or blurring, we want the network to reconstruct the original scan as similarly as possible with the input. Since the network does not know the mask, it should penalize the difference in distributions rather than shift in image. We assume the input distribution is symmetric.

(b) [3pt] Design a convolutional neural network of no more than 4 layers. Train it for at
least 10 epochs and plot both the training loss and testing

In [4]:
########################### BAD MODEL ################################
class KReconstruct(nn.Module):
    def __init__(self):
        super(KReconstruct, self).__init__()
        self.model = nn.Sequential(
            #nn.Conv2d(2, 4, kernel_size=12, stride=2, padding=0,dilation=1),  # 155x155
            #nn.BatchNorm2d(4),
            #nn.ReLU(),

            #nn.Conv2d(4, 16, kernel_size=7, stride=1, padding=0, dilation=2), # 143x143
            #nn.BatchNorm2d(16),
            #nn.ReLU(),

            #nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=0, dilation=1), # 139x139
            #nn.BatchNorm2d(32),
            #nn.ReLU(),

            #nn.ConvTranspose2d(32, 1, kernel_size=23, stride=2, padding=1, dilation=2, output_padding=1) # 320x320
            
            
            
            
            #nn.Conv2d(2,4, kernel_size=7, stride=1, dilation=2),  
            #nn.BatchNorm2d(4),
            #nn.ReLU(),
            
            #nn.Conv2d(4,8, kernel_size=5, stride=1, dilation=2),  
            #nn.BatchNorm2d(8),
            #nn.ReLU(),
            
            #nn.Conv2d(8,16, kernel_size=3, stride=1, dilation=1),  
            #nn.BatchNorm2d(16),
            #nn.ReLU(),
            
            #nn.ConvTranspose2d(16, 1, kernel_size=13, stride=1, padding=1, dilation=2, output_padding=0)
            
            
            
            
            #nn.Conv2d(1,4, kernel_size=5, stride=1, padding=4, dilation=2),
            #nn.BatchNorm2d(4),
            #nn.ReLU(),
            
            #nn.Conv2d(4,8, kernel_size=5, stride=1, padding=2, dilation=1),
            
            #nn.Conv2d(8,4, kernel_size=3, stride=1, padding=1, dilation=1),
            
            #nn.Conv2d(4,1, kernel_size=3, stride=1, padding=1, dilation=1)
            
            
            nn.Conv2d(1,8, kernel_size=5, stride=1, padding=2, dilation=1),
            nn.BatchNorm2d(8),
            nn.ELU(),
            
            nn.Conv2d(8,16, kernel_size=3, stride=1, padding=1, dilation=1),
            nn.BatchNorm2d(16),
            nn.ELU(),
            
            nn.Conv2d(16,1, kernel_size=3, stride=1, padding=1, dilation=1),
            nn.ReLU()
            
            #nn.Conv2d(4,1, kernel_size=3, stride=1, padding=1, dilation=1)
            
        )

    def forward(self, x):

        xabs = torch.abs(x)
        
        img = self.model(xabs)
        #out = nn.functional.log_softmax(img)
        
        return img#, out
    
    def test(self, x):
        #xreal = torch.real(x)
        #ximag = torch.imag(x)
        #x = torch.cat((xreal, ximag), 1)
        xabs = torch.abs(x)
        
        return self.model(xabs)


model = KReconstruct()
optimizer = torch.optim.Adam(model.parameters(), lr=0.5e-3, weight_decay=0.0005)
criterion = nn.MSELoss()


In [ ]:
kspace, M, gt = next(iter(test_loader))
m = KReconstruct()
this = m.test(acc_img(kspace).unsqueeze(1))
this.squeeze(1).size()

In [ ]:
# Begin training for 10 epochs

epochs=10
# set device
dtype = torch.float32
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('Running on: ', device)
model = model.to(device=device)
# keep track of losses
eval_dic = {'Loss_t': [], 'train_acc': [],'Loss_v': [], 'valid_acc': []}
loss_train = []
loss_val = []
loss_dict = {'train_loss':[], 'val_loss':[]}


print(f"\nTraining is about to start on {device}\n")
for e in range(epochs):
    print(f"\nTraining epoch {e}: \t")
    mia_loss = 0
    for batch_idx,(kspace, M, gt) in enumerate(tqdm(train_loader)):
        # prepare for training
        model.train(True)  
        optimizer.zero_grad()
        # move to gpu if available
        reconstructed_img = acc_img(kspace).to(device=device)
        #M = M.to(device=device)
        gt = gt.to(device=device)
        # forward pass
        out = model(reconstructed_img.unsqueeze(1))
        out = out.squeeze(1)
        # backpropagate loss
        loss = criterion(out, gt)
        loss = loss.to(torch.float32)
        loss.backward()
        mia_loss += loss.item()
        optimizer.step()
        
    # update loss dictionary
    loss_dict['train_loss'].append(mia_loss/(batch_idx+1))

    # print epoch loss
    print(f"\nEpoch training loss is {mia_loss/(batch_idx+1)}")

    with torch.no_grad():  # setting so that no backprop happens during validation
        model.eval()  # setting so that no backprop happens during validation
        vloss = 0
        for (kspace, M, gt) in tqdm(test_loader):
            reconstructed_img = acc_img(kspace).to(device=device)
            #M = M.to(device=device)
            gt = gt.to(device=device)
            out = model(reconstructed_img.unsqueeze(1))
            out = out.squeeze(1)
            vloss += criterion(out, gt).to(torch.float16).item()
        # update loss dictionary  
        loss_dict['val_loss'].append(vloss/len(test_loader))
        # print validation loss
        print(f"\nEpoch validation loss is {vloss/len(test_loader)}")

# save trained model
model_path = os.getcwd() + '\Modelex5ELU.pth'
torch.save(model.state_dict(), model_path)
    

In [ ]:
# Plot losses


#validation loss is actually test loss
plt.figure(figsize=(15,10),dpi=100)
plt.plot(range(epochs), loss_dict['train_loss'], 
         label='training loss', linewidth=5)
plt.plot(range(epochs), np.array(loss_dict['val_loss']), 
         label='test loss', linestyle='--', linewidth=5)
plt.ylabel('loss')
plt.xlabel('Epochs')
plt.grid()
plt.legend(loc=0, prop={'size': 20})


(c) [2pt] Repeat exercise 4c using your trained ConvNet.

In [ ]:
# Load model
model_path = os.getcwd() + '\Modelex5ELU.pth'
model = KReconstruct()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()
model.to('cpu')

kspace,Mask,gt = next(iter(test_loader))
# (mu,shrinkage,K,kspace,mask)
plt.figure(figsize=(15,20))
for i in range(5):

    rec_init_kp =acc_img(kspace[i:i+1,:,:]) # init recon from k-space
    output = model.forward(rec_init_kp.unsqueeze(1)) # recon mri_ista
    gt_MRI = gt[i,:,:]

    

    plt.subplot(3,5,i+1)
    plt.imshow(torch.abs(rec_init_kp.squeeze(0)),cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title("Init reconstruction")
    
    plt.subplot(3,5,i+6)
    plt.imshow(output.squeeze(0).squeeze(0).detach(),cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title("Model output")
    
    plt.subplot(3,5,i+11)
    plt.imshow(torch.abs(gt_MRI),cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title("Ground Truth")

    plt.tight_layout()

In [ ]:
model_path = os.getcwd() + '\Modelex5ELU.pth'
model = KReconstruct()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()
MSEtest=0
for (kspace,Mask,gt) in test_loader:
    reconstruct = acc_img(kspace)
    filteredwithCNN  = model.forward(reconstruct.unsqueeze(1)).squeeze(1)
    MSEtest += nn.functional.mse_loss(gt,filteredwithCNN).item()
print(f"Mean of the mse ON THE test set is: {MSEtest/len(test_loader)}")

## Exercise 6

We shall now combine the strenghts of ISTA with that of our deep learning ConvNet to
arrive ate some state-of-the-art results in reconstruction of accelerated MRI. To that end,
we shall be employing neural proximal gradient descent. This method is simmilar to LISTA,
but with some slight changes.

(b) [4pt] Implement neural proximal gradient descent for 5 unfolded iterations. As proximal
operator, make use of the ConvNet you designed in exercise 5. Train it for at
least 10 epochs and plot both the training loss and testing loss in terms of epochs or
batches seen.
hint: μ should remain between 0 and 1. Moreover, it helps to set a higher learning rate
for the μs compared to the other trainable parameters

In [7]:
model_path = os.getcwd() + '\Modelex5ELU.pth'
Proxi = KReconstruct()
Proxi.load_state_dict(torch.load(model_path))
Proxi.train(True)

class newISTA(nn.Module):
    def __init__(self, device):
        super(newISTA,self).__init__()   
        self.device=device
        self.mu1 = nn.parameter.Parameter(torch.ones(1)).to(device=self.device)
        #self.mu3 = nn.parameter.Parameter(torch.ones(1)).to(device=self.device)
        self.mu3 = torch.tensor(0.8, requires_grad=True)
        self.mu5 = nn.parameter.Parameter(torch.ones(1)).to(device=self.device)
        self.mu7 = nn.parameter.Parameter(torch.ones(1)).to(device=self.device)
        #self.mu9 = nn.parameter.Parameter(torch.ones(1)).to(device=self.device)
        #self.mu11 = nn.parameter.Parameter(torch.ones(1)).to(device=self.device)

        self.proxi1 = Proxi.to(device=self.device).train()

        self.proxi3 = KReconstruct().to(device=self.device).train()

        self.proxi5 = KReconstruct().to(device=self.device).train()
        
        self.proxi7 = KReconstruct().to(device=self.device).train()
        
        self.proxi9 = KReconstruct().to(device=self.device).train()
        
        #self.proxi11 = Proxi.to(device=self.device)


    def forward(self,x):
        x = torch.abs(x)
        x=x.to(device=self.device)
        
        eye = torch.eye(320)
        eye = eye.to(device=self.device)
        A = torch.abs(K_space(impulse))
        A = A.to(device=self.device)
        
        
        x1 = torch.matmul(A, x)
        i = self.proxi1(x1)
        x2 = torch.matmul(i, (eye-self.mu1*torch.matmul(torch.transpose(A, 0,1), A)))

        x3 = self.mu3 * torch.matmul(A, x)
        i = x2 + x3
        i = self.proxi3(i)
        x4 = torch.matmul(i, (eye-self.mu3*torch.matmul(torch.transpose(A, 0,1), A)))

        x5 = self.mu5 * torch.matmul(A, x)
        i = x4 + x5
        i = self.proxi5(i)
        x6 = torch.matmul(i, (eye-self.mu5*torch.matmul(torch.transpose(A, 0,1), A)))

        x7 = self.mu7 * torch.matmul(A, x)
        i = x6 + x7
        i = self.proxi7(i)
        #x8 = torch.matmul(i, (eye-self.mu7*torch.matmul(torch.transpose(A, 0,1), A)))
        
        #x9 = self.mu9 * torch.matmul(A, x)
        #i = x8 + x9
        #i = self.proxi9(i)
        #x10 = i @ (eye-self.mu9*A.T @ A)
        
        #x11 = self.mu11 * A @ x
        #i = x10 + x11
        #out = self.proxi11(i)
        print(f"\n mu3 is {self.mu3} and mu5 is {self.mu5}")
        out = self.proxi7(i)
        
        return out
    
    def test(self,x):
        return self.forward(x)

In [31]:
# test
kspace, M, gt = next(iter(test_loader))
m = newISTA(device='cpu')
m.eval()
this = m.test(acc_img(kspace).unsqueeze(1))
this.squeeze(1).size()

torch.Size([16, 320, 320])

In [8]:
# keep track of losses
loss_train = []
loss_val = []
loss_dict = {'train_loss':[], 'val_loss':[]}


# set device
dtype = torch.float32
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('Running on: ', device)


epochs=10
lamda = 0.3
K = 100
model = newISTA(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.5e-3, weight_decay=0.0005)
criterion = nn.MSELoss()


#mu*torch.matmul(y_flat,(A.T)) + (x_in@A-mu*x_in@A.T@A)

print(f"\nTraining is about to start on {device}\n")
for e in range(epochs):
    print(f"\nTraining epoch {e}: \t")
    mia_loss = 0
    for batch_idx,(kspace, M, gt) in enumerate(tqdm(train_loader)):
        # prepare for training
        model.train(True)  
        optimizer.zero_grad()
        # move to gpu if available
        #A = A.to(device=device)
        reconstructed_img = acc_img(kspace)
        #reconstructed_img = reconstructed_img.to(device=device)
        #M = M.to(device=device)
        gt = gt.to(device=device)
        # forward pass
        out = model(reconstructed_img.unsqueeze(1))
        out = out.squeeze(1)
        # backpropagate loss
        loss = criterion(out, gt)
        loss = loss.to(torch.float32)
        loss.backward()
        mia_loss += loss.item()
        optimizer.step()
        
    # update loss dictionary
    loss_dict['train_loss'].append(mia_loss/(batch_idx+1))

    # print epoch loss
    print(f"\nEpoch training loss is {mia_loss/(batch_idx+1)}")

    with torch.no_grad():  # setting so that no backprop happens during validation
        model.eval()  # setting so that no backprop happens during validation
        vloss = 0
        for (kspace, M, gt) in tqdm(test_loader):
            reconstructed_img = acc_img(kspace)
            reconstructed_img=reconstructed_img.to(device=device)
            #M = M.to(device=device)
            A = A.to(device=device)
            gt = gt.to(device=device)
            out = model(reconstructed_img.unsqueeze(1))
            out = out.squeeze(1)
            vloss += criterion(out, gt).to(torch.float16).item()
        # update loss dictionary  
        loss_dict['val_loss'].append(vloss/len(test_loader))
        # print validation loss
        print(f"\nEpoch validation loss is {vloss/len(test_loader)}")

# save trained model
model_path = os.getcwd() + '\ModelnewISTA.pth'
torch.save(model.state_dict(), model_path)
    

  0%|          | 0/47 [00:00<?, ?it/s]

Running on:  cuda

Training is about to start on cuda


Training epoch 0: 	


  0%|          | 0/47 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 100.00 MiB (GPU 0; 4.00 GiB total capacity; 2.47 GiB already allocated; 71.80 MiB free; 2.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [26]:
print(model.mu3)

tensor(0.8000, requires_grad=True)
